In [4]:
#This script takes every simulation with all points and calculates rmse and mae in each case

In [22]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [23]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param):
    VARS = ['x1',]
    x = dataframe[[c for c in VARS]].copy()
    y=dataframe.y

    if number_param==10:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')
    elif number_param==20:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np20.maxs200.2024-05-10 162907.551306.dat')

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dataframe)
    dplot['ybms'] = t.predict(x)
    dinterpolate=deepcopy(dplot)


    return dinterpolate
    

In [24]:
#Read NN and BMS data
functions=['tanh', 'leaky_ReLU'] #tanh, leaky_ReLU
realizations=2
N=9
sigmas=[0.0, 0.02, 0.04,0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.20]


runid=0
NPAR=10 #10, 20
steps=50000


rmse_nn_train=[];rmse_nn_test=[]
rmse_mdl_train=[];rmse_mdl_test=[]

mae_nn_train=[];mae_nn_test=[]
mae_mdl_train=[];mae_mdl_test=[]

n_index=[];r_index=[];sigma_index=[];function_index=[]

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for function in functions:
    for sigma in sigmas:
        for r in range(realizations+1):
            
            #Read NN data
            file_model='NN_no_overfit_' + function + '_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            file_model_hr='NN_no_overfit_' + function + '_sigma_' + str(sigma) + '_r_' + str(r) + '_res_0.01.csv'
            
            model_dhr='../data/inter_extrapolate_nns/' + file_model_hr
            dhr=pd.read_csv(model_dhr)

            model_d='../data/1x_resolution/trained_nns/' + file_model
            d=pd.read_csv(model_d)
            

            for n in range(N+1):
                n_index.append(n);r_index.append(r);sigma_index.append(sigma);function_index.append(function)

                #High resolution dataset
                dnhr=dhr[dhr['rep']==n]
                dnhr=dnhr[(dnhr['x1']>=-2.0) & (dnhr['x1']<=2.0)]
                dnhr=clean_index(dnhr)
                #-------------------------------------------------------------
                train_value=1.0                                                                               
                train_border_row=dnhr[(dnhr['x1']<=1.0) & (dnhr['x1']>=0.99)] 
                train_size=train_border_row.index[0]
                #-------------------------------------------------------------

                #Read BMS data
                filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
        
                trace=pd.read_csv('../data/MSTraces/1x_resolution/' + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])
                
                dinterpolate=add_bms_pred(dnhr, trace, NPAR)

                #Remove approximation points
                #-------------------------------------------------------------
                dinterpolate=dinterpolate.round({'x1':2})
                dn=d[d['rep']==n]
                dn.set_index('Unnamed: 0', inplace=True)
                dn.index.name = None
                dn=dn.reset_index(drop=True)
                dn=dn.round({'x1':2})
                dinterpolate=dinterpolate[-dinterpolate['x1'].isin(dn['x1'].tolist() )] #cross dataframes to get only high resolution points
                #display(dinterpolate)
                #-------------------------------------------------------------
                
                #Errors
                #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                rmse_nn_train_i=root_mean_squared_error(dinterpolate.loc[:train_size-1]['ymodel'],dinterpolate.loc[:train_size -1]['y'])
                rmse_nn_train.append(rmse_nn_train_i)
                
                rmse_nn_test_i=root_mean_squared_error(dinterpolate.loc[train_size-1:]['ymodel'],dinterpolate.loc[train_size -1:]['y'])
                rmse_nn_test.append(rmse_nn_test_i)

                mae_nn_train_i=mean_absolute_error(dinterpolate.loc[:train_size-1]['ymodel'],dinterpolate.loc[:train_size -1]['y'])
                mae_nn_train.append(mae_nn_train_i)
            
                mae_nn_test_i=mean_absolute_error(dinterpolate.loc[train_size-1:]['ymodel'],dinterpolate.loc[train_size -1:]['y'])
                mae_nn_test.append(mae_nn_test_i)


                try:
                    rmse_mdl_train_i=root_mean_squared_error(dinterpolate.loc[:train_size-1]['ybms'],dinterpolate.loc[:train_size-1]['y'])
                except ValueError:
                    rmse_mdl_train_i=np.inf
                rmse_mdl_train.append(rmse_mdl_train_i)

                try:
                    rmse_mdl_test_i=root_mean_squared_error(dinterpolate.loc[train_size-1:]['ybms'],dinterpolate.loc[train_size-1:]['y'])
                except ValueError:
                    rmse_mdl_test_i=np.inf
                    
                rmse_mdl_test.append(rmse_mdl_test_i)

                try:
                    mae_mdl_train_i=mean_absolute_error(dinterpolate.loc[:train_size-1]['ybms'],dinterpolate.loc[:train_size -1]['y'])
                except ValueError:
                    mae_mdl_train_i=np.inf
                    
                mae_mdl_train.append(mae_mdl_train_i)

                try:
                    mae_mdl_test_i=mean_absolute_error(dinterpolate.loc[train_size-1:]['ybms'],dinterpolate.loc[train_size -1:]['y'])
                except ValueError:
                    mae_mdl_test_i=np.inf
                
                mae_mdl_test.append(mae_mdl_test_i)
                #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~              
             

              
      
errors_df=pd.DataFrame({'sigma':sigma_index, 'function':function_index, 'mae_nn_interp.':mae_nn_train, 'mae_nn_extrap.':mae_nn_test, 
                        'mae_mdl_interp.':mae_mdl_train, 'mae_mdl_extrap.':mae_mdl_test, 'rmse_nn_interp.':rmse_nn_train, 
                        'rmse_nn_extrap.': rmse_nn_test, 'rmse_mdl_interp.':rmse_mdl_train, 'rmse_mdl_extrap.': rmse_mdl_test, 
                        'n':n_index, 'r': r_index})
errors_df.to_csv('../data/all_errors_interpolation' + '.csv')
display(errors_df)

,sigma,function,mae_nn_interp.,mae_nn_extrap.,mae_mdl_interp.,mae_mdl_extrap.,rmse_nn_interp.,rmse_nn_extrap.,rmse_mdl_interp.,rmse_mdl_extrap.,n,r
0,0.0,tanh,0.038834,0.048822,0.001137,0.009393,0.047852,0.052995,0.001340,0.011199,0,0
1,0.0,tanh,0.011743,0.121524,0.002827,0.309184,0.015463,0.127937,0.003454,0.454497,1,0
2,0.0,tanh,0.041361,0.555351,0.000997,0.056736,0.111692,0.560637,0.001222,0.069993,2,0
3,0.0,tanh,0.012957,0.091136,0.000314,0.041855,0.033141,0.101108,0.000406,0.048799,3,0
4,0.0,tanh,0.008928,0.047356,0.000165,0.002835,0.010469,0.049501,0.000248,0.003921,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
655,0.2,leaky_ReLU,0.063666,0.285437,0.008551,0.142148,0.095268,0.309775,0.010784,0.164822,5,2
656,0.2,leaky_ReLU,0.189519,1.391914,0.124834,0.230120,0.307740,1.400525,0.142086,0.234066,6,2
657,0.2,leaky_ReLU,0.100138,0.040536,0.035055,0.042514,0.127973,0.047088,0.041496,0.046699,7,2
658,0.2,leaky_ReLU,0.105994,0.105083,0.102439,0.162222,0.127666,0.105542,0.114628,0.176435,8,2
